In [2]:
%load_ext autoreload
%autoreload 2

# Compute engine stand alone

In [153]:
file = joblib.load('schemas.barfi')

In [154]:
from_client = joblib.load('_from_client.pkl')

In [155]:
from blocks import init
from streamlit import session_state as session
session.dados = pd.DataFrame()
init()

2024-07-21 08:44:48.305 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`


import blocks
starting blocks


In [156]:
from Barfi import ComputeEngine

def get_execution(fluxo, base_blocks_list):
    _ce = ComputeEngine(blocks=base_blocks_list)
    _ce.add_editor_state(fluxo)
    _ce._map_block_link()
    _ce._execute_compute()

In [157]:
get_execution(file['soma3'], session.blocks)

Resultado: 3


In [161]:
joblib.dump(session['blocks'], 'blocks.pkl')

['blocks.pkl']

# Entrada block

In [162]:
file = joblib.load('schemas.barfi')
entr = file['entradas']

In [164]:
get_execution(entr, session.blocks)

nome: entrada-1 valor: 1
nome: entrada-2 valor: 2
Resultado: 3


In [165]:
def get_dict_from_interfaces(interface):
    aux = {}
    for i in interface:
        aux[i[0]] = i[1]
    return aux


def encode_dict_to_interface(aux_dict):
    interface = [[key, aux_dict[key]] for key in aux_dict]
    return interface


def encode_val_to_entradas(entra, val_dict):
    aux = entra.copy()
    for entrada in aux:
        name = entrada['name']
        tmp = get_dict_from_interfaces(entrada['interfaces'])
        tmp['entrada']['value'] = val_dict[name]
        encoded_interface = encode_dict_to_interface(tmp)
        entrada['interfaces'] = encoded_interface
    return aux

def get_ids_entradas(entradas):
    aux = entradas.copy()
    ids = []
    for entrada in aux:
        tmp = get_dict_from_interfaces(entrada['interfaces'])
        ids.append(tmp['entrada']['id'])
    return ids


def get_ids_from_conn(connections, to_ids):
    aux = connections.copy()
    entry_dict = {}
    ids = []
    ids_from = []
    pos = []
    for i, conn in enumerate(connections):
        if conn['to'] in to_ids:
            entry_dict[i] = {'id': conn['id'],
                             'from': conn['from'],
                             'to': conn['to']}
    return entry_dict


def encode_entradas_to_const(entra, val_dict):
    aux = entra['nodes'].copy()
    for entrada in aux:
        name = entrada['name']
        if name in val_dict:
            entrada['type'] = 'constante'
            entrada['options'] = [['valor', val_dict[name]]]
            tmp = get_dict_from_interfaces(entrada['interfaces'])
            del tmp['entrada']
            encoded_interface = encode_dict_to_interface(tmp)
            entrada['interfaces'] = encoded_interface


def get_to_dict_from_conn(connection):
    aux = {i['to']:i for i in connection}
    return aux


def encode_conn_from_dict(dict_conn):
    return list(dict_conn.values())


def delete_connections(fluxo, to_ids):
    tmp = get_to_dict_from_conn(fluxo['connections'])
    for i in to_ids:
        try:
            del tmp[i]
        except:
            pass
    return tmp


def get_entradas(fluxo):
    entradas = [i for i in fluxo['nodes'] if i['type'] == 'entrada']
    return entradas


def get_fluxo_const(fluxo, val_dict):
    aux_fluxo = deepcopy(fluxo)
    saidas = [i for i in aux_fluxo['nodes'] if i['type'] == 'saida']
    saida_names = [i['name'] for i in saidas]
    entradas = [i for i in aux_fluxo['nodes'] if i['type'] == 'entrada']
    entrada_names = [i['name'] for i in entradas]
    ids_entradas = get_ids_entradas(entradas)
    entry_conn = get_ids_from_conn(aux_fluxo['connections'], ids_entradas)
    tmp = delete_connections(aux_fluxo, ids_entradas)
    aux_fluxo['connections'] = encode_conn_from_dict(tmp)
    encode_entradas_to_const(aux_fluxo, val_dict)
    return aux_fluxo


In [166]:
val_dict = {'entrada-1': 2, 'entrada-2': 3}
# tmp = entradas.copy()
# new_entr = encode_val_to_entradas(tmp, val_dict)

In [167]:
new = get_fluxo_const(entr, val_dict)

In [168]:
get_execution(entr, session.blocks)

nome: entrada-1 valor: 1
nome: entrada-2 valor: 2
Resultado: 3


In [169]:
get_execution(new, session.blocks)

Resultado: 5


In [170]:
file['teste']

{'nodes': [],
 'connections': [],
 'panning': {'x': 36.37855271680658, 'y': -60.82564451658715},
 'scaling': 0.8334151762422278}